In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')

In [ ]:
# Load the synthetic energy dataset
df = pd.read_csv('../data/synthetic/energy_dataset.csv')
print(f"Dataset shape: {df.shape}")
df.head()

In [ ]:
# Basic statistics
df.describe()

## 1. Correlation Analysis

In [ ]:
# Calculate correlations with energy
numeric_cols = df.select_dtypes(include=[np.number]).columns
correlations = df[numeric_cols].corr()['energy_kwh'].sort_values(ascending=False)

print("Feature correlations with energy consumption:")
print(correlations)

In [ ]:
# Plot correlation heatmap
fig, ax = plt.subplots(figsize=(10, 8))
correlation_matrix = df[numeric_cols].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, ax=ax, fmt='.2f')
plt.title('Feature Correlation Heatmap')
plt.tight_layout()
plt.show()

## 2. Energy Distribution Analysis

In [ ]:
# Create complexity categories
df['complexity_category'] = pd.cut(
    df['avg_word_length'],
    bins=[0, 5.0, 6.0, 7.0, float('inf')],
    labels=['Simple', 'Moderate', 'Complex', 'Very Complex']
)

# Energy distribution by complexity
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
ax1 = axes[0]
df['energy_kwh'].hist(bins=20, ax=ax1, edgecolor='black')
ax1.set_xlabel('Energy (kWh)')
ax1.set_ylabel('Frequency')
ax1.set_title('Energy Consumption Distribution')

# Box plot by complexity
ax2 = axes[1]
df.boxplot(column='energy_kwh', by='complexity_category', ax=ax2)
ax2.set_xlabel('Complexity Category')
ax2.set_ylabel('Energy (kWh)')
ax2.set_title('Energy by Prompt Complexity')
plt.suptitle('')

plt.tight_layout()
plt.show()

## 3. Feature Impact on Energy

In [ ]:
# Scatter plots of key features vs energy
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Token count vs Energy
axes[0, 0].scatter(df['token_count'], df['energy_kwh'], alpha=0.6)
axes[0, 0].set_xlabel('Token Count')
axes[0, 0].set_ylabel('Energy (kWh)')
axes[0, 0].set_title('Token Count vs Energy')

# Avg word length vs Energy
axes[0, 1].scatter(df['avg_word_length'], df['energy_kwh'], alpha=0.6, color='orange')
axes[0, 1].set_xlabel('Avg Word Length')
axes[0, 1].set_ylabel('Energy (kWh)')
axes[0, 1].set_title('Word Length vs Energy')

# Num layers vs Energy
axes[1, 0].scatter(df['num_layers'], df['energy_kwh'], alpha=0.6, color='green')
axes[1, 0].set_xlabel('Number of Layers')
axes[1, 0].set_ylabel('Energy (kWh)')
axes[1, 0].set_title('Model Layers vs Energy')

# Training hours vs Energy
axes[1, 1].scatter(df['training_hours'], df['energy_kwh'], alpha=0.6, color='red')
axes[1, 1].set_xlabel('Training Hours')
axes[1, 1].set_ylabel('Energy (kWh)')
axes[1, 1].set_title('Training Hours vs Energy')

plt.tight_layout()
plt.show()

## 4. Statistical Summary by Model Configuration

In [ ]:
# Group by num_layers and analyze
layer_analysis = df.groupby('num_layers').agg({
    'energy_kwh': ['mean', 'std', 'min', 'max'],
    'token_count': 'mean',
    'avg_word_length': 'mean'
}).round(4)

print("Energy Statistics by Model Layer Configuration:")
layer_analysis

## 5. Key Findings

### Observations:
1. **Token count** has the strongest correlation with energy consumption
2. **Average word length** (proxy for complexity) shows moderate positive correlation
3. **Model layers** contribute to higher energy usage
4. Prompts with fewer than 20 tokens typically consume under 1.0 kWh

### Recommendations:
- Keep prompts concise (< 50 tokens when possible)
- Use simpler vocabulary for routine queries
- Choose smaller models for straightforward tasks
- Monitor complexity scores to identify optimization opportunities

In [ ]:
# Calculate potential savings
high_energy = df[df['energy_kwh'] > df['energy_kwh'].median()]
low_energy = df[df['energy_kwh'] <= df['energy_kwh'].median()]

print(f"High energy prompts (above median):")
print(f"  - Count: {len(high_energy)}")
print(f"  - Avg tokens: {high_energy['token_count'].mean():.1f}")
print(f"  - Avg word length: {high_energy['avg_word_length'].mean():.2f}")
print(f"\nLow energy prompts (at or below median):")
print(f"  - Count: {len(low_energy)}")
print(f"  - Avg tokens: {low_energy['token_count'].mean():.1f}")
print(f"  - Avg word length: {low_energy['avg_word_length'].mean():.2f}")

potential_savings = (high_energy['energy_kwh'].mean() - low_energy['energy_kwh'].mean()) / high_energy['energy_kwh'].mean() * 100
print(f"\nPotential energy savings by optimizing high-energy prompts: {potential_savings:.1f}%")